In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sys
import math
from math import cos,atan,floor 
import cv2 as cv
import numpy as np
import os
from sklearn import mixture
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score

In [18]:
# functions :

def load_image(subdir, file):
    
    img = cv2.imread(os.path.join(subdir, file),0)

    return img


def height_and_width(img):
    
    height = len(img)
    width = len(img[0])
    
    return height,width


def dimention_mesure_of(height,width):
    
    dimention_mesure=max(height,width)
    
    return dimention_mesure


def show_image_in_shape(img):
    
    cv2.namedWindow('',cv2.WINDOW_NORMAL)
    cv2.imshow('', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def number_of_clusters(img):
    
    img.reshape(-1, 1)
    X = []
    for i in range(len(img)):# height
        for j in range(len(img[0])):# width
            point = img[i,j]
            if point != 0:
                X.append([i])
    X = np.array(X)
    
    plt.hist(X[:,0],200, range=[0, 800], facecolor='gray', align='mid')
    plt.show()
    
    X = X.reshape(-1,1)
    
    sil_coeff_max = 0
    n_cluster_max = 1
    cluster_errors = []
    
    for n_cluster in range(1, 10):
        kmeans = KMeans(n_clusters=n_cluster,max_iter=10).fit(X)
        label = kmeans.labels_
        sil_coeff = silhouette_score(X, label, metric='euclidean')
        if sil_coeff_max < sil_coeff :
            n_cluster_max = n_cluster
            sil_coeff_max = sil_coeff
        cluster_errors.append( kmeans.inertia_ )
        print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))
    plt.plot(cluster_errors)
    plt.show()
    
    number_of_clusters = n_cluster_max
    
    return number_of_clusters

    
def morphologie_step(img,dimention_mesure):
    
    img =  cv2.bitwise_not(img)
    img = cv2.GaussianBlur(img, (1, 1), 1)
    
    #kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (4, 4))
    #img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel,iterations=2)
    kernel =  cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel,iterations=1)
    
    labelnum, labelimg, contours, GoCs= cv2.connectedComponentsWithStats(img,cv2.CC_STAT_AREA  )
    
    surfaces_of_components=[]
    firstmaxs=[]
    secondmaxs=[]
    for label in range(1,labelnum):
        x,y = GoCs[label]
        img = cv2.circle(img, (int(x),int(y)), 1, (0,0,255), -1)
        x,y,w,h,size = contours[label]
        img = cv2.rectangle(img, (x,y), (x+w,y+h), (255,255,0), 1)
        surfaces_of_components.append(min(w,h)**2)
    
    if surfaces_of_components:
        firstmax = max(surfaces_of_components)/dimention_mesure
        firstmaxs.append(firstmax)
        surfaces_of_components.remove(np.array(max(surfaces_of_components)))
        if surfaces_of_components:
            secondmax = max(surfaces_of_components)/dimention_mesure
        else:
            secondmax = 0
            secondmaxs.append(secondmax)
    else:
        firstmax = 0
        secondmax = 0
    
    value_sum = 0
    
    for value in firstmaxs:
        value_sum = value_sum + (value**2)
    
    cv2.imshow('', img)
    cv2.waitKey() 
    
    return firstmax,secondmax,value_sum


def circle_step(img,dimention_mesure):
    
    img = cv2.medianBlur(img,5)
    
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                    param1=50,param2=20,minRadius=0,maxRadius=0)
    
    circles = np.uint16(np.around(circles))
    print circles
    
    num_circles = len(circles[0])
    
    if num_circles:
        firstmaxcircle = max(circles[0][:,2])/dimention_mesure
    
    sum_circle=0
    
    #img = cv2.circle(img, (int(x),int(y)), 1, (0,0,255), -1)
    #cv2.imshow('', img)
    #cv2.waitKey()
    for value in circles[0][:,2]:
        sum_circle = sum_circle +  value**2
        
    return num_circles,firstmaxcircle,sum_circle

def line_step(img):
    
    # Loads an image
    src = img

    # Check if image is loaded fine
    if src is None:
        print ('Error opening image!')
        print ('Usage: hough_lines.py [image_name -- default ' + default_file + '] \n')
        return -1
    
    height = len(src)
    width = len(src[0])
    
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    src = cv2.morphologyEx(src, cv2.MORPH_OPEN, kernel,iterations=2)
    #kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    #src = cv2.morphologyEx(src, cv2.MORPH_CLOSE, kernel,iterations=1)
    
    #edges :
    #dst = cv.Canny(src, 50, 200, None, 3)
    dst = cv.Canny(src, 100, 100, None, 3)
    
    # Copy edges to the images that will display the results in BGR
    cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)
    
    lines = cv.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)

    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
    
    linesP = cv.HoughLinesP(dst, 1, np.pi / 180, 50, None, 0.25*50, 5*10)
    
    list_longeurs = []
    list_incline = []
    
    if linesP is not None:
        for i in range(0, len(linesP)):

            l = linesP[i][0]
            list_longeurs.append( math.sqrt( (l[3]-l[1])**2 + (l[2]-l[0])**2 ) / (max(height,width,1)) )
            list_incline.append( math.fabs(cos(2*(math.atan( (l[3]-l[1])/max(l[2]-l[0],1) ))+(math.pi/2)))  )
            cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)
    
    #cv.imshow("Source", src)
    #cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
    #cv.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdstP)
    
    #cv.waitKey(0)

    incline=0
    for i in range(len(list_incline)):
        incline = incline +  list_incline[i]
        
    return incline



def filter_features(n_cluster_max, firstmax,secondmax,value_sum,num_circles,firstmaxcircle,sum_circle,incline,img):
    scale = 1
    
    if(n_cluster_max<3 and  firstmax <5 and  secondmax <4 and value_sum<30*scale
            and num_circles < 150*scale and firstmaxcircle<1 and sum_circle<(10**6)*scale 
        and incline<2*scale) :
        print(' Number_of_clusters : ' , number_of_clusters)
        print(' Firstmax : ' , 1*firstmax)
        print(' Secondmax : ' , 1*secondmax)
        print(' Value_sum : ' , 1*value_sum)
        print(' Num_circles :' , num_circles)
        print(' Firstmaxcircle :' , firstmaxcircle)
        print(' Sum_circle : ' , sum_circle)
        print(' Incline : ' , incline)
        print(' File : ' , file)   
        print()



        
        #cv2.imshow('', img)
        #cv2.waitKey() 

In [ ]:
rootdir = '/media/cuong-nguyen/B0EA64A4EA646894/backup_ubuntu/2018/Projet/rentre_signee/Sourcecode/rentree_signee_v1.0/py-faster-rcnn/lib/process/detect_signature/Signature/signature_no_square/signature_yes'

k=0
nombre_limit = 150
running_min = 10**10
running_max = 0
for subdir, dirs, files in os.walk(rootdir):
    #print files
    for file in files:
        print file
        if k < nombre_limit :
            # modify file :
            #file = 'cut5_modifier10.png'
            
            # load step :
            #print('Load step : ')
            img = load_image(subdir, file)
            
            # compute height and width :
            height,width = height_and_width(img)
            
            # compute dimention_mesure :
            dimention_mesure = dimention_mesure_of(height,width)
            

            # morphologie step :
            #print('Morphologie step : ')
            firstmax,secondmax,value_sum = morphologie_step(img,dimention_mesure)
            
            # cercle step :
            #print('Cercle step : ')
            num_circles,firstmaxcircle,sum_circle= circle_step(img,dimention_mesure)

            # line step :
            #print('Line step : ')
            incline = line_step(img)
            
            # point group step :
            #print('Group step : ')
            #n_cluster_max = point_group_step(subdir,file)
            #print('Filter step : ')
            
            # filter step :
            filter_features(1,firstmax,secondmax,value_sum,num_circles,firstmaxcircle,sum_circle,incline,img)
            
            #print('End of step :' ,k)
            
            if running_min > value_sum :
                running_min = value_sum
            
            if running_max < value_sum :
                running_max = value_sum
            
            k=k+1
print('running_min  : ' , running_min)
print('running_max :' , running_max)

CRMSF17536147_{17525680-79F0-4BFD-882D-F869C8F5EA57}.tif-5_C420.png
CRMSF17540369_{92AEA0E7-56DE-490A-87DF-F9B55D5CFB3C}.tif-4_C420.png
CRMSF17557163_{DF7F600F-6482-4D5A-BA10-BA25B6FB5840}.tif-9_C420.png
